In [1]:
from scipy.interpolate import interp1d
import pycbc.noise
import pycbc.psd
import pycbc.waveform
import pylab
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.mlab as mlab
import numpy as np
import seaborn

from scipy import signal
from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz

#
# Define PSD
#
def aLIGO_PSD(flow = 10.0, FMAX=8000, delta_f = 1./16., SEED=None):
    flen = int(FMAX / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    return psd

def aLIGO_PSD_inp(flow = 10.0, FMAX=8000, delta_f = 1./16., SEED=None):
    flen = int(FMAX / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    #psd = pycbc.psd.analytical.aLIGOZeroDetHighPower(flen, delta_f, flow)
    freqs = psd.sample_frequencies
        
    ## have to tweak the data to avoid 1/0 problem
    sel = np.where( psd.data<1.e-60 )
    psd.data[sel] = 1
        
    psd_interp = interp1d(freqs, psd)

    if 0:
        plt.loglog(freqs, np.sqrt(psd) )
        plt.show()
    
    return psd_interp

def LIGO_PSD_inp(FMAX=8192, delta_f = 1./16.):
    flen = int(FMAX / delta_f) + 1
    freqs = np.linspace(0,FMAX,flen)
    Pxx = (1.e-22*(18./(0.1+freqs))**2)**2+0.7e-23**2+((freqs/2000.)*4.e-23)**2
    
    psd_interp = interp1d(freqs, Pxx)
    return psd_interp

def PSD_from(timeseries, RATE=8192, alpha=0.1):
    ## calculate the PSD of data
    NFFT=RATE     
    ss_psd, f_ss = mlab.psd(timeseries, Fs = RATE, NFFT = NFFT, noverlap=NFFT/8, window=signal.tukey(NFFT, alpha=alpha) )
    psd_inp = interp1d(f_ss, ss_psd)
    
    return f_ss, psd_inp


from scipy.signal import butter, filtfilt, iirdesign, zpk2tf, freqz

def whiten(strain, interp_psd, dt, bp=0):
    Nt = len(strain)
    f = np.fft.rfftfreq(Nt, dt)
    #print len(f), f
    
    #dwindow = 1  ##signal.tukey(Nt, alpha=alpha)
    hf = np.fft.rfft(strain)
    
    norm = np.sqrt(dt*2)
    white_hf = hf / np.sqrt(interp_psd(f)) * norm
    white_ht = np.fft.irfft(white_hf, n=Nt)
    
    if bp:
        ## Bandpassing with 4th-order Butterworth D/A filter
        fband = [10.,800.]
        bb, ab = butter(4, [fband[0]*2./RATE, fband[1]*2./RATE], btype='band')
        normalization = np.sqrt((fband[1]-fband[0])/(RATE/2.0))
        white_ht = filtfilt(bb, ab, white_ht) / normalization
        
        if 0:  ## TODO
            plt.figure()
            w, h = signal.freqs(bb, ab)
            plt.plot(w, 20 * np.log10(abs(h)))
            plt.xscale('log')
            plt.title('Butterworth filter frequency response')
            plt.xlabel('Frequency [radians / second]')
            plt.ylabel('Amplitude [dB]')
            plt.margins(0, 0.1)
            plt.grid(which='both', axis='both')
            plt.axvline(100, color='green') # cutoff frequency
            plt.show()

    return white_ht
            

/home/lincy/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
###
###  I made two train/test datasets with sampling rate 8192/4096:
###     bbh_test_xxxx.h5
###     bbh_train_xxxx.h5
###

###  Update: I realized the whiten noise is exactly the Gaussian(0,1), so don't need to prepare it. 
### 
###
import h5py
RATE = 0   ## to be read from data file

aligo_psd   = aLIGO_PSD()
f_aligo_psd = aligo_psd.sample_frequencies
aligo_psd_inp = aLIGO_PSD_inp()

def get_whiten_template(h5name, A=1.0):
  
    HDF5_FILE = h5name
    f = h5py.File(HDF5_FILE,'r')

    RATE = f["/waveform"].attrs.get('srate')
    dt=1.0/RATE
    keys = f["/waveform"].keys()
    #print(keys)
    
    co=0
    xhc = []
    xhp = []
    y1 = []
    y2 = []
    n = []
    for i in keys:
        i = int(i)
        key = 'waveform/%d'% i
        kp  = 'waveform/%d/hp'%i
        kc  = 'waveform/%d/hc'%i
    
        m1 = f[key].attrs['m'][0]
        m2 = f[key].attrs['m'][1]
        #s1   = f[key].attrs['sz'][0]
        #flow = f[key].attrs['F_low']
        midx = f[key].attrs['midx']
        hp  = f[kp][:]
        hc  = f[kc][:]

        if (co%100==0):  
            print ("process ...", m1, m2, co, i)
            print (midx)
        co=co+1
    
        #leng = len(hp)
        #maxh = max(np.abs(hp+hc*1.j))
        #print ("MAX: ", maxh)
 
        ## to slightly move peak randomly, find the starting index in the template
        W=RATE*2   ## fix 2-sec strain
        merger_idx = W * 0.66    ###  put merger at 66% position of template
        
        #idxm = np.where( t2m >= 0 )[0][0]                         ## locate the index of merger time
        
        y1.append( m1 )
        y2.append( m2 )

        #==========
        #for h in [hp, hc]:   ## generate 2 instances for each data
            
        #idx0 = int(midx - W * np.random.uniform(0.55, 0.7)  + 1)  
        idx0 = int(midx - merger_idx + 1)
        if (idx0<0): idx0 = 0                                    ## if the template is too short, let it be.  

        # Generate LIGO noise
        #ligo_ns = pycbc.noise.noise_from_psd(W, dt, aLIGO_PSD_imp, seed=None)

        # Injected / shifted signal
        minidx = min(len(hp)-1, idx0+W)

        shifted_hp = np.zeros(W)   ##ligo_ns.data
        shifted_hc = np.zeros(W)   ##ligo_ns.data

        shifted_hp[0:minidx-idx0] = shifted_hp[0:minidx-idx0] + hp[idx0:minidx]
        shifted_hc[0:minidx-idx0] = shifted_hc[0:minidx-idx0] + hc[idx0:minidx]

        whiten_hp = whiten(   shifted_hp, aligo_psd_inp, dt)
        whiten_hc = whiten(   shifted_hc, aligo_psd_inp, dt)

        tmphp =  whiten_hp[RATE/2:-RATE/2]
        tmphc =  whiten_hc[RATE/2:-RATE/2]

        maxf = A * np.sqrt(2*np.log(2.)) / max( np.sqrt(tmphp**2+ tmphc**2)  )
        
        xhp.append( tmphp * maxf )   ### normalize to the scale of gaussian noise (0,1)
        xhc.append( tmphc * maxf )
    
    f.close()  
    return xhp, xhc, y1,y2

import h5py as h5
import numpy as np
save = h5.File("white_h_fixed.h5", "w")  #_small
save.attrs['merger_idx'] = 0
save.attrs['srate'] = 8192

for tag in ['train', 'val', 'test']:
    xhp, xhc, y1, y2  = get_whiten_template("bbh_8192_dm1_%s.h5"%tag)   ##0.07

    save.create_dataset("%s_hp"%tag,  data=xhp, dtype='f')  # compression='gzip'
    save.create_dataset("%s_hc"%tag,  data=xhc, dtype='f')  # compression='gzip'
    save.create_dataset("%s_m1"%tag,  data=y1, dtype='f')
    save.create_dataset("%s_m2"%tag,  data=y2, dtype='f')

save.close()


('process ...', 23, 52, 0, 1124)
20980
('process ...', 30, 44, 100, 1466)
21421
('process ...', 37, 50, 200, 1773)
21274
('process ...', 38, 66, 300, 1828)
20840
('process ...', 25, 27, 400, 1204)
21906
('process ...', 46, 69, 500, 2107)
20835
('process ...', 7, 27, 600, 121)
21251
('process ...', 44, 54, 700, 2029)
21170
('process ...', 53, 59, 800, 2286)
21026
('process ...', 26, 43, 900, 1271)
21416
('process ...', 18, 41, 1000, 833)
21283
('process ...', 16, 23, 1100, 696)
22085
('process ...', 10, 17, 1200, 309)
22299
('process ...', 22, 50, 1300, 1068)
21029
('process ...', 18, 38, 1400, 830)
21432
('process ...', 37, 38, 1500, 1761)
21528
('process ...', 62, 69, 1600, 2467)
20812
('process ...', 28, 58, 1700, 1385)
20903
('process ...', 13, 37, 1800, 524)
21203
('process ...', 21, 76, 1900, 1039)
19737
('process ...', 12, 38, 2000, 461)
21048
('process ...', 29, 57, 2100, 1432)
20967
('process ...', 16, 68, 2200, 741)
19644
('process ...', 13, 19, 2300, 506)
22243
('process ...'